In [ ]:
'''
!!!!!!! Эта ячейка для запуска сборщика мозаик на Google Drive через Google Colab !!!!!!!

'''

from google.colab import drive
drive.mount('/content/drive')
# Определение директорий исходных изображений и назначения для мозаики
src_dir = '/content/drive/MyDrive/UII/Capture/'
dest_dir = '/content/drive/MyDrive/UII/Capture/Mosaic/'

In [20]:
'''
!!!!!!! Эта ячейка для запуска сборщика мозаик на локальной машине через Jupyter Notebook (или IDE) !!!!!!!

'''

# Определение директорий исходных изображений и назначения для мозаики
src_dir = r'C:\Users\anton\UII\AZavod_Ural'
dest_dir = r'C:\Users\anton\UII\AZavod_Ural\Mosaic'

In [23]:
'''
!!!!!!! Собственно сам сборщик мозаик !!!!!!!

'''

import os
import shutil
import random
from tqdm import tqdm
from PIL import Image
import concurrent.futures

# Функция для удаления директории
def delete_directory(path: str):
    # Удаляет директорию по заданному пути, если она существует
    if os.path.exists(path):
        try:
            shutil.rmtree(path)
        except OSError as e:
            print(f'[ERROR] Error deleting a directory: `{path}`: {e.strerror}')

# Функция для создания директории
def ensure_directory_exists(directory):
    # Создает директорию, если она не существует
    if not os.path.exists(directory):
        os.makedirs(directory)

# Функция для масштабирования изображения в размер полотна
def process_image(file, src_dir, min_image, canvas_size):
    path = os.path.join(src_dir, file)
    try:
        with Image.open(path) as img:
            if min(img.width, img.height) >= min_image:                       
                img.thumbnail((canvas_size, canvas_size), Image.Resampling.LANCZOS)
                return img.copy()
    except Exception as e:
        print(f"Ошибка при открытии изображения {file}: {e}")
    return None

def create_mosaic(src_dir, dest_dir, canvas_size=640, min_image=40):

    # Создание списка файлов для обработки
    image_files = [file for file in os.listdir(src_dir) if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff'))]
    cnt_img_all = len(image_files)
    all_images = []

    # Запуск многопоточной предобработки
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Подача задач на обработку изображений
        future_to_image = {executor.submit(process_image, file, src_dir, min_image, canvas_size): file for file in image_files}
        for future in tqdm(concurrent.futures.as_completed(future_to_image), total=len(image_files), desc="Предобработка директории", unit=" files", leave=True):
            img = future.result()
            if img is not None:
                all_images.append(img)

    if not all_images:
        print("В папке нет изображений.")
        return

    # Сортировка изображений по площади для последующего выбора первого элемента на полотно
    all_images.sort(key=lambda img: img.width * img.height, reverse=True)
    
    cnt_img = len(all_images)
    num_canvas = 0
    with tqdm(total=cnt_img, desc="Распределение картинок по мозаикам", unit=" files", leave=True) as pbar:
        while all_images:
            # Создание нового полотна
            canvas = Image.new('RGB', (canvas_size, canvas_size), (0, 0, 0))
            x, y = 0, 0
            next_row_y = 0            
            filled_rows_width = []  # Для хранения ширины каждого заполненного ряда

            # Размещение изображений на текущем полотне
            
            occupied_areas = []  # Список занятых областей
            # Размещение первого элемента на полотно
            largest_img = all_images.pop(0)
            canvas.paste(largest_img, (x, y))
            occupied_areas.append({'left': x, 'top': y, 'right': x + largest_img.width, 'bottom': y + largest_img.height})
            filled_rows_width.append(largest_img.width)
            x += largest_img.width
            next_row_y = max(next_row_y, y + largest_img.height)
            pbar.update(1)
                           
            while all_images:           
                img = random.choice(all_images)

                # Проверяем, можно ли поместить изображение в текущее положение, без наложения
                if x + img.width <= canvas_size and y + img.height <= canvas_size and \
                        all(not (x < area['right'] and x + img.width > area['left'] and 
                                y < area['bottom'] and y + img.height > area['top']) for area in occupied_areas):
                    
                    canvas.paste(img, (x, y))
                    occupied_areas.append({'left': x, 'top': y, 'right': x + img.width, 'bottom': y + img.height})
                    x += img.width
                    filled_rows_width.append(x)
                    next_row_y = max(next_row_y, y + img.height)
                    all_images.remove(img)
                    pbar.update(1)

                    # Перебор оставшихся изображений для дозаполнения ряда
                    extra_images_to_try = len(all_images)
                    while extra_images_to_try > 0 and all_images:
                        extra_img = random.choice(all_images)
                        if x + extra_img.width <= canvas_size and y + extra_img.height <= canvas_size and \
                                all(not (x < area['right'] and x + extra_img.width > area['left'] and 
                                        y < area['bottom'] and y + extra_img.height > area['top']) for area in occupied_areas):

                            canvas.paste(extra_img, (x, y))
                            occupied_areas.append({'left': x, 'top': y, 'right': x + extra_img.width, 'bottom': y + extra_img.height})
                            x += extra_img.width
                            filled_rows_width.append(x)
                            next_row_y = max(next_row_y, y + extra_img.height)
                            all_images.remove(extra_img)
                            pbar.update(1)
                        extra_images_to_try -= 1
                else:
                    # Переход к новому ряду или полотну
                    if y + next_row_y >= canvas_size:
                        break
                    x = 0
                    y = next_row_y

            # Расчет смещения для центровки мозаики на полотне
            max_filled_width = max(filled_rows_width, default=0)
            offset_x = (canvas_size - max_filled_width) // 2
            offset_y = (canvas_size - next_row_y) // 2
            final_canvas = Image.new('RGB', (canvas_size, canvas_size),(0, 0, 0))
            final_canvas.paste(canvas, (offset_x, offset_y))
            # Сохранение готового полотна в указанной директории
            num_canvas += 1
            final_canvas.save(os.path.join(dest_dir, f'mosaic_{num_canvas}.jpg'))

    # Вывод количества созданных полотен
    print(f"\n{cnt_img} картинок из исходных {cnt_img_all} размещены на {len(os.listdir(dest_dir))} полотнах\n{cnt_img_all-cnt_img} картинок отброшены из-за малого размера")

# Удаление предыдущей директории мозаики, если она существует
delete_directory(dest_dir)
# Создание новой директории для мозаики
ensure_directory_exists(dest_dir)
# Вызов функции создания мозаики
create_mosaic(src_dir, dest_dir, 640, 40)

Распределение картинок по мозаикам: 100%|██████████| 1351/1351 [00:07<00:00, 178.31 files/s]


1351 картинок из исходных 1351 размещены на 820 полотнах
0 картинок отброшены из-за малого размера
